# 🦟 Advanced Dengue Fever Heatmap: Puerto Rico (2024 CDC Data)
This notebook creates a rich, interactive heatmap of Dengue fever cases in Puerto Rico using real CDC data. It includes:
- Heatmap with custom gradient
- Scaled circle markers
- Interactive popups and tooltips
- Multiple basemaps
- Fullscreen mode and layer control
- Fixed error related to gradient key type

In [2]:
# Step 1: Install required packages
!pip install folium pandas

In [3]:
# Step 2: Import libraries
import pandas as pd
import folium
from folium.plugins import HeatMap, Fullscreen

In [46]:
# Step 3: Load the Dengue case data from CSV and ensure clean types
df = pd.read_csv('dengue_puerto_rico_2024.csv')
df['Latitude'] = pd.to_numeric(df['Latitude'], errors='coerce')
df['Longitude'] = pd.to_numeric(df['Longitude'], errors='coerce')
df['Cases'] = pd.to_numeric(df['Cases'], errors='coerce')
df = df.dropna(subset=['Latitude', 'Longitude', 'Cases'])
df

,Municipality,Latitude,Longitude,Cases
0,San Juan,18.4655,-66.1057,1200
1,Carolina,18.3808,-65.9574,354
2,Rincón,18.3400,-67.2500,252
3,Lares,18.2950,-66.8771,240
4,Bayamón,18.3985,-66.1557,233


In [48]:
# Step 4: Create the interactive map
m = folium.Map(location=[18.22, -66.5], zoom_start=9, tiles=None)

# Add multiple tile layers
folium.TileLayer('CartoDB positron', name='Light').add_to(m)
folium.TileLayer('Stamen Toner', name='Dark').add_to(m)
folium.TileLayer('OpenStreetMap', name='Standard').add_to(m)

# Fullscreen control
Fullscreen(position='topright').add_to(m)

In [50]:
# Step 5: Add custom heatmap with gradient (string keys to avoid error)
gradient = {
    '0.2': 'blue',
    '0.4': 'lime',
    '0.6': 'orange',
    '0.8': 'red'
}

heat_data = [[row['Latitude'], row['Longitude'], row['Cases']] for _, row in df.iterrows()]
HeatMap(heat_data, radius=25, blur=15, gradient=gradient, name='Dengue Heatmap').add_to(m)

In [52]:
# Step 6: Add scaled circle markers
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=row['Cases'] / 100,
        color='crimson' if row['Cases'] > 500 else 'darkblue',
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(f"<strong>{row['Municipality']}</strong><br>Dengue Cases: {row['Cases']}", max_width=250),
        tooltip=row['Municipality']
    ).add_to(m)

In [14]:
# Step 7: Add map title and layer control
title_html = '''
<h3 align='center' style='font-size:20px'><b>Dengue Fever Incidence in Puerto Rico (2024)</b></h3>'''
m.get_root().html.add_child(folium.Element(title_html))

folium.LayerControl().add_to(m)
m

In [ ]:
#The interactive heatmap visualization reveals significant geographic clustering of Dengue fever ##cases across Puerto Rico in 2024, 
#with notably higher concentrations in urban and coastal regions. 
#These patterns may reflect a combination of environmental factors, population density, 
#and infrastructure disparities that influence mosquito breeding and virus transmission.

#This spatial analysis highlights the importance of targeted public health interventions, particularly in high-risk areas. 
#By integrating geospatial data with epidemiological insights, this tool can support more proactive and 
#localized responses to vector-borne disease outbreaks.

